In [1]:
import pandas as pd
import numpy as np
import pickle
import time
from datetime import date

import matplotlib.pyplot as plt
import seaborn as sns

### Data cleaning functions

In [2]:
def column_rename(df):
    df.rename(columns={'srch_ci':'check_in', 'srch_co':'check_out', 'srch_adults_cnt':'adult_count', \
                         'srch_children_cnt':'child_count', 'srch_rm_cnt':'room_count',\
                         'srch_destination_id':'destination_id', 'srch_destination_type_id':'destination_type_id', \
                        'cnt':'similar_events'}, inplace=True)

In [3]:
def feature_engine(df):
    df['date_time'] = pd.to_datetime(df['date_time'])
    df['check_in'] = pd.to_datetime(df['check_in'], errors='coerce')
    df['check_out'] = pd.to_datetime(df['check_out'], errors='coerce')
    df['stay_duration'] = (df['check_out'] - df['check_in']).astype('timedelta64[D]')
    df['plan_time'] = (df['check_in'] - df['date_time']).astype('timedelta64[D]')
#     df['day_of_week'] = df['date_time'].dt.day_name()

In [4]:
def fillna(df):
    df['orig_destination_distance'] = df['orig_destination_distance'].astype(np.float64)
    df['orig_destination_distance'].fillna((df['orig_destination_distance'].mean()), inplace=True)
    df['stay_duration'].fillna((df['stay_duration'].mean()), inplace=True)
    df['plan_time'].fillna((df['plan_time'].mean()), inplace=True)

### Train 2m dataset

In [5]:
train = pd.read_csv('train_2m.csv')

In [6]:
train.head()

,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,...,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,is_booking,cnt,hotel_continent,hotel_country,hotel_market,hotel_cluster
0,2014-08-11 07:46:59,2,3,66,348,48862,2234.2641,12,0,1,...,0,1,8250,1,0,3,2,50,628,1
1,2014-08-11 08:22:12,2,3,66,348,48862,2234.2641,12,0,1,...,0,1,8250,1,1,1,2,50,628,1
2,2014-08-11 08:24:33,2,3,66,348,48862,2234.2641,12,0,0,...,0,1,8250,1,0,1,2,50,628,1
3,2014-08-09 18:05:16,2,3,66,442,35390,913.1932,93,0,0,...,0,1,14984,1,0,1,2,50,1457,80
4,2014-08-09 18:08:18,2,3,66,442,35390,913.6259,93,0,0,...,0,1,14984,1,0,1,2,50,1457,21


In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1999999 entries, 0 to 1999998
Data columns (total 24 columns):
date_time                    object
site_name                    int64
posa_continent               int64
user_location_country        int64
user_location_region         int64
user_location_city           int64
orig_destination_distance    float64
user_id                      int64
is_mobile                    int64
is_package                   int64
channel                      int64
srch_ci                      object
srch_co                      object
srch_adults_cnt              int64
srch_children_cnt            int64
srch_rm_cnt                  int64
srch_destination_id          int64
srch_destination_type_id     int64
is_booking                   int64
cnt                          int64
hotel_continent              int64
hotel_country                int64
hotel_market                 int64
hotel_cluster                int64
dtypes: float64(1), int64(20), object(3)
m

In [8]:
# clean up
column_rename(train)
feature_engine(train)
fillna(train)

In [9]:
# filter the top ten hotel cluster only
most_common_clusters = list(train.hotel_cluster.value_counts().head(10).index)
train = train.loc[train['hotel_cluster'].isin(most_common_clusters)]

In [10]:
# drop stay duration < 0
train = train.drop(train[train['stay_duration'] < 0].index)

In [11]:
train.head()

,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,...,destination_id,destination_type_id,is_booking,similar_events,hotel_continent,hotel_country,hotel_market,hotel_cluster,stay_duration,plan_time
6,2014-07-16 09:42:23,2,3,66,189,10067,2014.665587,501,0,0,...,8267,1,0,2,2,50,675,41,1.0,15.0
7,2014-07-16 09:45:48,2,3,66,189,10067,2014.665587,501,0,1,...,8267,1,0,1,2,50,675,41,1.0,15.0
9,2014-07-16 09:55:24,2,3,66,189,10067,2014.665587,501,0,0,...,8267,1,0,1,2,50,675,70,1.0,15.0
10,2014-07-16 10:00:06,2,3,66,189,10067,2014.665587,501,0,0,...,8267,1,0,1,2,50,675,98,1.0,15.0
43,2014-11-22 20:55:38,30,4,195,991,47725,2014.665587,1048,1,0,...,8803,1,0,1,3,151,1197,5,2.0,215.0


### Feature Engineering

In [12]:
# solo trip or not 
train['solo_travel'] = np.where((train['adult_count']==1) & (train['child_count']==0),1,0)
# short trip using stay duration
train['short_trip'] = np.where((train['stay_duration'] <= 3),1,0)

In [13]:
# with open('train_round1.pickle', 'wb') as to_write:
#     pickle.dump(train, to_write)

In [14]:
# train.to_csv('train_round1.csv')

In [15]:
# weekend trip
train['check_in_day'] = train['check_in'].dt.day_name()
train['check_out_day'] = train['check_out'].dt.day_name()

train['weekend_trip'] = np.where((train['stay_duration']<=3) & (train['check_in_day']=='Friday'), 1,0)

train.drop(columns=['check_in_day', 'check_out_day'], inplace=True)

In [16]:
train.columns

Index(['date_time', 'site_name', 'posa_continent', 'user_location_country',
       'user_location_region', 'user_location_city',
       'orig_destination_distance', 'user_id', 'is_mobile', 'is_package',
       'channel', 'check_in', 'check_out', 'adult_count', 'child_count',
       'room_count', 'destination_id', 'destination_type_id', 'is_booking',
       'similar_events', 'hotel_continent', 'hotel_country', 'hotel_market',
       'hotel_cluster', 'stay_duration', 'plan_time', 'solo_travel',
       'short_trip', 'weekend_trip'],
      dtype='object')

In [17]:
# train.to_csv('train_round2.csv')

In [18]:
#frequency of booking
booking_rate = train.groupby('user_id').is_booking.mean()
booking_rate = booking_rate.reset_index()
booking_rate.rename(columns={'is_booking':'booking_rate'}, inplace=True)
booking_rate.head()

,user_id,booking_rate
0,17,0.0
1,22,0.0
2,45,0.0
3,102,0.0
4,103,0.0


In [19]:
train = pd.merge(train,booking_rate, on='user_id')

In [20]:
# train.to_csv('train_round3.csv')

In [21]:
#view only
train['price_compare'] = np.where((train['booking_rate']==0),1,0)

In [22]:
#business trip
# train.groupby(['user_id','adult_count', 'weekend_trip'])['is_booking'].mean()
train['biz_trip'] = np.where((train['adult_count']==1) & (train['weekend_trip']==0) & (train['is_booking']==1),1,0)

In [23]:
train.to_csv('train_round5.csv')

In [24]:
# remove datetime columns
# train.drop(columns=['date_time', 'check_in','check_out'], inplace=True)

In [25]:
#for merging purpose
train.rename(columns={'destination_id':'srch_destination_id'}, inplace=True)

In [26]:
train[['hotel_cluster','srch_destination_id']]
# train.groupby(['hotel_cluster'])['srch_destination_id'].count()
train['srch_destination_id'].nunique()

8868

### Fill up info of location

In [33]:
train['user_location_country'].replace([66,205,1,215], ['USA','Canada','Italy','Mexico',], inplace=True)
train['user_location_region'].replace([246, 174, 348], ['Hawaii','California','New York'], inplace=True)
train['user_location_city'].replace([48862,24103,26232],['New York City','Los Angeles','San Francisco'], inplace=True)
train['hotel_country'].replace([8,50,70,198,204],['Mexico','USA','UK','Canada','France'], inplace=True)
train['hotel_market'].replace([27,19,110,675,701,628,365,1230,637,191,623,212,213,214],\
                              ['Paris','London','Cancun','NYC','Miami','Las Vegas','Los Angeles','San Francisco',\
                              'Chicago','Washington','Philadelphia','Hawaii','Hawaii','Hawaii'], inplace=True)
train['hotel_continent'].replace([2,6],['North America','Europe'], inplace=True)

In [41]:
train.columns

Index(['date_time', 'site_name', 'posa_continent', 'user_location_country',
       'user_location_region', 'user_location_city',
       'orig_destination_distance', 'user_id', 'is_mobile', 'is_package',
       'channel', 'check_in', 'check_out', 'adult_count', 'child_count',
       'room_count', 'srch_destination_id', 'destination_type_id',
       'is_booking', 'similar_events', 'hotel_continent', 'hotel_country',
       'hotel_market', 'hotel_cluster', 'stay_duration', 'plan_time',
       'solo_travel', 'short_trip', 'weekend_trip', 'booking_rate',
       'price_compare', 'biz_trip'],
      dtype='object')

### Destinations

In [28]:
destinations = pd.read_csv('destinations.csv')

In [29]:
destinations.head()

,srch_destination_id,d1,d2,d3,d4,d5,d6,d7,d8,d9,...,d140,d141,d142,d143,d144,d145,d146,d147,d148,d149
0,0,-2.198657,-2.198657,-2.198657,-2.198657,-2.198657,-1.897627,-2.198657,-2.198657,-1.897627,...,-2.198657,-2.198657,-2.198657,-2.198657,-2.198657,-2.198657,-2.198657,-2.198657,-2.198657,-2.198657
1,1,-2.181690,-2.181690,-2.181690,-2.082564,-2.181690,-2.165028,-2.181690,-2.181690,-2.031597,...,-2.165028,-2.181690,-2.165028,-2.181690,-2.181690,-2.165028,-2.181690,-2.181690,-2.181690,-2.181690
2,2,-2.183490,-2.224164,-2.224164,-2.189562,-2.105819,-2.075407,-2.224164,-2.118483,-2.140393,...,-2.224164,-2.224164,-2.196379,-2.224164,-2.192009,-2.224164,-2.224164,-2.224164,-2.224164,-2.057548
3,3,-2.177409,-2.177409,-2.177409,-2.177409,-2.177409,-2.115485,-2.177409,-2.177409,-2.177409,...,-2.161081,-2.177409,-2.177409,-2.177409,-2.177409,-2.177409,-2.177409,-2.177409,-2.177409,-2.177409
4,4,-2.189562,-2.187783,-2.194008,-2.171153,-2.152303,-2.056618,-2.194008,-2.194008,-2.145911,...,-2.187356,-2.194008,-2.191779,-2.194008,-2.194008,-2.185161,-2.194008,-2.194008,-2.194008,-2.188037


In [30]:
# latent description of destination
# srch_destination_id = train_df.destination_id

In [31]:
# reduce dimensions of destination folder
from sklearn.decomposition import PCA

pca = PCA(n_components=10)
dest_small = pca.fit_transform(destinations[["d{0}".format(i + 1) for i in range(149)]])
dest_small = pd.DataFrame(dest_small)
dest_small["destination_id"] = destinations["srch_destination_id"]

In [32]:
dest_small.head()

,0,1,2,3,4,5,6,7,8,9,destination_id
0,-0.044268,0.169419,0.032522,-0.014278,-0.069688,-0.012327,-0.169851,0.173756,-0.201333,0.083768,0
1,-0.440761,0.077405,-0.091572,-0.020233,0.013217,0.029683,-0.013471,0.007504,-0.031848,-0.028691,1
2,0.001033,0.020677,0.012108,0.134132,0.142035,0.088833,-0.143639,0.089091,0.172453,-0.038045,2
3,-0.480467,-0.040345,-0.019320,-0.040100,-0.027394,0.017350,0.029529,0.033157,-0.026493,0.042428,3
4,-0.207253,-0.042694,-0.011744,-0.017518,-0.019871,-0.037966,-0.036658,-0.004237,-0.007555,0.009955,4
